In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#### - Напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [2]:
def hypothesis(theta, X):
    """
    Гіпотеза лінійної регресії у векторному вигляді.

    Параметри:
    theta : numpy array
        Вектор параметрів (ваг).
    x : numpy array
        Вектор ознак.

    Повертає:
    float
        Прогнозоване значення за гіпотезою.
    """
    return np.dot(X, theta)

#### - Cтворіть функцію для обчислення функції втрат у векторному вигляді;

In [3]:
def compute_cost(theta, X, y):
    """
    Обчислення функції втрат у векторному вигляді.

    Параметри:
    theta : numpy array
        Вектор параметрів (ваг).
    X : numpy array
        Матриця ознак (кожен рядок - окремий приклад, кожний стовпчик - окрема ознака).
    y : numpy array
        Вектор цільових значень.

    Повертає:
    float
        Значення функції втрат.
    """
    m = len(y)  # Кількість прикладів у наборі даних
    predictions = hypothesis(theta, X)  # Прогнозовані значення
    square_error = (predictions - y) ** 2  # Квадрат помилок
    cost = (1 / (2 * m)) * np.sum(square_error)  # Загальна функція втрат
    return cost

#### - Реалізуйте один крок градієнтного спуску;

In [4]:
def gradient_descent_step(theta, X, y, learning_rate):
    """
    Виконує один крок градієнтного спуску.

    Параметри:
    theta : numpy array
        Поточні значення параметрів моделі (ваг).
    X : numpy array
        Матриця ознак (кожен рядок - окремий приклад, кожний стовпчик - окрема ознака).
    y : numpy array
        Вектор цільових значень.
    learning_rate : float
        Крок навчання (швидкість навчання).

    Повертає:
    numpy array
        Оновлені значення параметрів моделі.
    """
    m = len(y)  # Кількість прикладів у наборі даних
    predictions = hypothesis(theta, X)  # Прогнозовані значення
    errors = predictions - y  # Різниця між прогнозами та справжніми значеннями
    gradient = (1 / m) * np.dot(X.T, errors)  # Градієнт функції втрат
    theta -= learning_rate * gradient  # Оновлення параметрів
    return theta

#### - Знайдіть найкращі параметри w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [5]:
df = pd.read_csv('Housing.csv')
print(df.head(5))

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [6]:
# Перетворення ознак та цільового вектору у відповідний формат
X = np.array(df[['area','bedrooms','bathrooms']])
y = np.array(df['price']) / 1000000

# Нормалізуємо дані
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Додаємо колонку з одиницями для розрахунку зсуву (intercept)
X_scaled = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

# Розділяємо дані на тренувальний та тестувальний набори
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Початкові значення параметрів
w = np.zeros(X_train.shape[1])  # 4, оскільки ми маємо 3 ознаки (площа, кількість ванних кімнат та кількість спалень) + 1 додатковий стовпець
learning_rate = 0.01  # Швидкість навчання
num_iterations = 1000  # Кількість ітерацій градієнтного спуску

# Градієнтний спуск
for _ in range(num_iterations):
    # Один крок градієнтного спуску
    w = gradient_descent_step(w, X_train, y_train, learning_rate)
    
# Оцінка параметрів w
print("Оптимальні параметри w:", w)

# Обчислення функції втрат для оптимальних параметрів
cost = compute_cost(w, X_train, y_train)
print("Значення функції втрат:", cost)

# Перевірка точності на тестовому наборі
predictions = np.dot(X_test, w)
mse = np.mean((predictions - y_test) ** 2)
print("Середня квадратична помилка на тестовому наборі:", mse)

Оптимальні параметри w: [4.73598472 0.74923517 0.26672061 0.71257199]
Значення функції втрат: 0.7803690039876643
Середня квадратична помилка на тестовому наборі: 2.7496785497395297


#### - Знайдіть ці ж параметри за допомогою аналітичного рішення;

In [7]:
def find_parameters_analytically(X, y):
    """
    Знаходить оптимальні параметри моделі лінійної регресії аналітичним рішенням.
    
    Параметри:
        X (numpy.ndarray): Матриця ознак.
        y (numpy.ndarray): Вектор цільових значень.
    
    Повертає:
        numpy.ndarray: Вектор параметрів моделі.
    """
    X_transpose = np.transpose(X)
    w = np.linalg.inv(X_transpose.dot(X)).dot(X_transpose).dot(y)
    return w

parameters = find_parameters_analytically(X_train, y_train)
print("Оптимальні параметри w:", parameters)

# Перевірка точності на тестовому наборі
predictions = np.dot(X_test, parameters)
mse = np.mean((predictions - y_test) ** 2)
print("Середня квадратична помилка на тестовому наборі:", mse)

Оптимальні параметри w: [4.73630316 0.74902305 0.26560486 0.71401645]
Середня квадратична помилка на тестовому наборі: 2.7500404793090523


In [8]:
# Поділ даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення та навчання моделі
regressor = LinearRegression().fit(X_train, y_train)

# Отримання оптимальних параметрів (w)
optimal_parameters = regressor.coef_

# Отримання прогнозів для тестового набору
predictions = regressor.predict(X_test)

# Отримання значення функції втрат (середньоквадратична помилка) на тестових даних
cost = mean_squared_error(y_test, predictions)

print("Оптимальні параметри w:", optimal_parameters)
print("Середня квадратична помилка на тестовому наборі:", cost)

Оптимальні параметри w: [3.45466570e-04 3.60197650e-01 1.42231966e+00]
Середня квадратична помилка на тестовому наборі: 2.7500404793090514


#### - Порівняйте отримані результати.

В усіх проведених трьох розрахунках середня квадратична помилка на тестовому наборі приблино однакова і дорівнює 2.75. В перших 2х розрахунках з врахуванням параметра зсуву, оптимальні параметри w теж близькі за значенням